# GANomaly Notebook for Individual Experiment Results

## Initial Configurations

### Libraries import

In [1]:
import os
import sys
import cv2
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from ipywidgets import interact, IntSlider
from IPython.display import display, clear_output

sys.path.append("../")\

2022-08-03 01:42:42.924667: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
from utils.metrics import accuracy, precision, recall, specificity, f1_score
from utils.metrics import dagostinoPearson_test, andersonDarling_test, shapiroWilks_test, chiSquare_test, fOneWay_test
from utils.metrics import brownForsythe_test, levene_test, bartlett_test
from utils.metrics import mannWhitney_test, kruskalWallis_test, kolmogorovSmirnov_test
from utils.savers import generate_qq_plot
from utils.common import format_index

### Functions definition

In [3]:
def precision_recall_curve(y_true, y_pred, num_thresholds=200):
    precisions = []
    recalls = []
    thresholds = np.linspace(np.min(y_pred), np.max(y_pred), num_thresholds)
    for t in thresholds:
        tp = np.count_nonzero(np.logical_and(y_true, (y_pred > t)))
        fp = np.count_nonzero(np.logical_and(np.logical_not(y_true), (y_pred > t)))
        fn = np.count_nonzero(np.logical_and(y_true, (y_pred <= t)))
        if tp+fp == 0:
            precisions.append(0)
        else:
            precisions.append(precision(tp, fp))
        if tp + fn == 0:
            recalls.append(0)
        else:
            recalls.append(recall(tp, fn))
    return np.r_[precisions], np.r_[recalls], thresholds

### Experiments selection

In [4]:
# experiments_ids = [
#     "0028", "0032", "0034", "0035", # 3D B32 BN Control
#     "0038", "0042", "0047", "0052", # 2D B32 BN Control
#     "0029", "0033", "0036", "0037", # 3D B16 BN Control
#     "0039", "0043", "0050", "0055", # 2D B16 BN Control
#     "0044", "0046", "0048", "0051", # 3D B16 BN Parkinson
#     "0063", "0067", "0069", "0070", # 2D B16 BN Parkinson
#     "0040", "0041", "0045", "0049", # 3D B16 RGB Control
#     "0053", "0058", "0061", "0066", # 2D B16 RGB Control
#     "0059", "0062", "0065", "0068", # 3D B16 RGB Parkinson
#     "0054", "0057", "0060", "0064", # 2D B16 RGB Parkinson
# ]
experiments_ids = [format_index(i) for i in range(1, 6)]
# experiments_ids = ["0029", '0033', '0036', '0037']
save_path_2_excels = "../results/Results_Reports/"

exps = {}
root_path = "../results/Ganomaly_3D/0003_eval_of_0001/"
for exp_id in experiments_ids:
    for i in sorted(os.listdir(root_path)):
        if exp_id in i:
            exp_path = os.path.join(root_path, i)
            exps[exp_id] = {
                "path": exp_path
            }
exps

{'0001': {'path': '../results/Ganomaly_3D/0003_eval_of_0001/0001_Ganomaly3D-64x64x64x1'},
 '0002': {'path': '../results/Ganomaly_3D/0003_eval_of_0001/0002_Ganomaly3D-64x64x64x1'},
 '0003': {'path': '../results/Ganomaly_3D/0003_eval_of_0001/0003_Ganomaly3D-64x64x64x1'},
 '0004': {'path': '../results/Ganomaly_3D/0003_eval_of_0001/0004_Ganomaly3D-64x64x64x1'},
 '0005': {'path': '../results/Ganomaly_3D/0003_eval_of_0001/0005_Ganomaly3D-64x64x64x1'}}

## Standar metrics excel

### Errors loading

In [5]:
for exp_id in experiments_ids:
    base_path = os.path.join(exps[exp_id]["path"], "outputs/errors/")
    for t in ["encoder", "contextual", "adversarial"]:
            for c in ["normal", "abnormal"]:
                exps[exp_id]["all_{}_{}".format(t, c)] = np.r_[[]]
            
    for t in ["encoder", "contextual", "adversarial"]:
        for m in ["train", "test"]:
            if m == "train":
                if os.path.isfile(os.path.join(base_path, t, m, "normal.npy")):
                    classes = ["normal"] 
                else:
                    classes = ["abnormal"]
            else:
                classes = ["normal", "abnormal"]

            for c in classes:
                all_data = "all_{}_{}".format(t, c)
                errors = np.load(os.path.join(base_path, t, m, c + ".npy"))
                exps[exp_id]["{}_{}_{}".format(m, t, c)] = errors
                exps[exp_id][all_data] = np.concatenate([exps[exp_id][all_data], errors])

In [6]:
for exp_id in experiments_ids:
    base_path = os.path.join(exps[exp_id]["path"], "outputs/latent_vectors/input_generator")
    for t in ["encoder", "contextual", "adversarial"]:
        for m in ["train", "test"]:
            if m == "train":
                if "train_encoder_normal" in exps[exp_id].keys():
                    classes = ["normal"] 
                else:
                    classes = ["abnormal"]
            else:
                classes = ["normal", "abnormal"]
            for c in classes:
                patients_ids_positions = [
                    int(i.split("_")[1].split("-")[1].split(".")[0]) for i in sorted(
                        os.listdir(os.path.join(base_path, m, c))
                    )
                ]
                data = "{}_{}_{}".format(m, t, c)
                key = "{}_{}".format(data, "patients")
                exps[exp_id][key] = {}
                
                for p_id in np.unique(patients_ids_positions):
                    exps[exp_id][key][p_id] = []
                
                for i, p_id in enumerate(patients_ids_positions):
                    exps[exp_id][key][p_id].append(exps[exp_id][data][i])

### Building excel with standard metrics

In [ ]:
data_table = []
data_columns = ["Exp ID", "L_gen", "L_disc", "Acc (t=0.5)", "Pre_orig", 
                "Rec_orig", "Spe_orig", "F1_orig", "AUC", "Threshold", 
                "Acc_thre", "Pre_thre", "Rec_thre", "Spe_thre", "F1_thre",
                "Homo Level"
               ]
for exp_id in experiments_ids:
    experiment_folder = exps[exp_id]["path"]
    
    train_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/train.csv"))
    test_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/test.csv"))

    group = "encoder"
    data = "test_{}_".format(group)
    errors = ["normal", "abnormal"]
    y_true = np.concatenate([[i]*exps[exp_id][data + j].shape[0] for i,j in enumerate(errors)]) 
    y_pred = np.concatenate([exps[exp_id][data+i] for i in errors])
    
    if "train_{}_normal".format(group) in exps[exp_id].keys():
        y_pred = (y_pred - np.min(y_pred)) / (np.max(y_pred) - np.min(y_pred))
        test_class = "normal"
    else:
        y_pred = 1 - ((y_pred - np.min(y_pred)) / (np.max(y_pred) - np.min(y_pred)))
        test_class = "abnormal"
    
    precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred)
    deltas_pre_4_rec = np.abs(precisions - recalls)
    threshold = thresholds[np.argmin(deltas_pre_4_rec[deltas_pre_4_rec != 0])]

    TP = tf.keras.metrics.TruePositives(threshold)
    TN = tf.keras.metrics.TrueNegatives(threshold)
    FP = tf.keras.metrics.FalsePositives(threshold)
    FN = tf.keras.metrics.FalseNegatives(threshold)
    AUC = tf.keras.metrics.AUC()

    TP.update_state(y_true, y_pred)
    TN.update_state(y_true, y_pred)
    FP.update_state(y_true, y_pred)
    FN.update_state(y_true, y_pred)
    AUC.update_state(y_true, y_pred)
    
    homo_key = "{}{}_patients".format(data, test_class)
    homo_metric = 0
    for p_id in exps[exp_id][homo_key]:
        homo_4_pat = int(brownForsythe_test(
            sorted(exps[exp_id][homo_key][p_id]), sorted(exps[exp_id]["train_{}_{}".format(group, test_class)])
        )) + int(levene_test(
            sorted(exps[exp_id][homo_key][p_id]), sorted(exps[exp_id]["train_{}_{}".format(group, test_class)])
        )) + int(bartlett_test(
            sorted(exps[exp_id][homo_key][p_id]), sorted(exps[exp_id]["train_{}_{}".format(group, test_class)])
        ))
        homo_metric += homo_4_pat/3
    homo_metric /= len(exps[exp_id][homo_key])
    
    data_table.append([
        exp_id,
        train_metrics.loc[train_metrics.shape[0] - 1 ,"gen_error"],
        train_metrics.loc[train_metrics.shape[0] - 1 ,"disc_error"],
        test_metrics.loc[test_metrics.shape[0] - 1 ,"accuracy"],
        test_metrics.loc[test_metrics.shape[0] - 1 ,"precision"],
        test_metrics.loc[test_metrics.shape[0] - 1 ,"recall"],
        test_metrics.loc[test_metrics.shape[0] - 1 ,"specificity"],
        test_metrics.loc[test_metrics.shape[0] - 1 ,"f1_score"],
        AUC.result().numpy(),
        threshold,
        np.max(accuracy(TP.result().numpy(), TN.result().numpy(), FP.result().numpy(), FN.result().numpy())),
        np.max(precision(TP.result().numpy(), FP.result().numpy())),
        np.max(recall(TP.result().numpy(), FN.result().numpy())),
        np.max(specificity(TN.result().numpy(), FP.result().numpy())),
        np.max(f1_score(TP.result().numpy(), FP.result().numpy(), FN.result().numpy())),
        homo_metric
    ])

pd.DataFrame(data_table, columns=data_columns).to_excel(os.path.join(save_path_2_excels, "standard_metrics.xlsx"), index=False)

In [14]:
# Testing homocedasticity methods
#group = "contextual"
exp_id = "0001"
test_class = "normal"
compare_group = "train"

for group in ["encoder", "contextual", "adversarial"]:
    data = "train_{}_".format(group)
    homo_key = "{}{}_patients".format(data, test_class)
    homo_metric = 0
    for p_id in exps[exp_id][homo_key]:
        homo_4_pat = int(brownForsythe_test(
            sorted(exps[exp_id][homo_key][p_id]), sorted(exps[exp_id]["{}_{}_{}".format(compare_group, group, test_class)])
        )) + int(levene_test(
            sorted(exps[exp_id][homo_key][p_id]), sorted(exps[exp_id]["{}_{}_{}".format(compare_group, group, test_class)])
        )) + int(bartlett_test(
            sorted(exps[exp_id][homo_key][p_id]), sorted(exps[exp_id]["{}_{}_{}".format(compare_group, group, test_class)])
        ))
        homo_metric += homo_4_pat/3
#         print(homo_4_pat)
    homo_metric /= len(exps[exp_id][homo_key])
    print(homo_metric)

0.3055555555555555
0.027777777777777776
0.08333333333333333


In [18]:
for t in ["encoder", "contextual", "adversarial"]:
        for c in ["normal", "abnormal"]:
            globals()["total_{}_{}".format(t, c)] = np.r_[[]]

for exp_id in experiments_ids:
    for t in ["encoder", "contextual", "adversarial"]:
#         for m in ["train", "test"]:
#             if m == "train":
#                 if "train_encoder_normal" in exps[exp_id].keys():
#                     classes = ["normal"] 
#                 else:
#                     classes = ["abnormal"]
#             else:
#                 classes = ["normal", "abnormal"]
        m = "test"
        for c in ["normal", "abnormal"]:
            total_data = "total_{}_{}".format(t, c)
            globals()[total_data] = np.concatenate([
                globals()[total_data], 
                exps[exp_id]["{}_{}_{}".format(m, t, c)]
            ])

In [20]:
total_encoder_abnormal.shape

(313,)

In [ ]:
exps['0037']['test_encoder_abnormal'].shape

In [ ]:
# total_encoder_abnormal = np.array([0]*88, dtype=np.float64)
# for i in experiments_ids:
#     total_encoder_abnormal += exps[i]["test_encoder_abnormal"]
# total_encoder_abnormal /= len(experiments_ids)
# total_encoder_abnormal = exps['0036']["test_encoder_abnormal"]
total_encoder_abnormal = np.concatenate([
    exps['0029']["test_encoder_abnormal"][:24],
    exps['0033']["test_encoder_abnormal"][24:48],
    exps['0036']["test_encoder_abnormal"][48:72],
    exps['0037']["test_encoder_abnormal"][72:88]
])
total_encoder_abnormal.shape

In [62]:
group = "encoder"
data = "total_{}_".format(group)
errors = ["normal", "abnormal"]
y_true = np.concatenate([[i]*globals()[data + j].shape[0] for i,j in enumerate(errors)]) 
y_pred = np.concatenate([globals()[data + i] for i in errors])

precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred)
deltas_pre_4_rec = np.abs(precisions - recalls)
threshold = thresholds[np.argmin(deltas_pre_4_rec[deltas_pre_4_rec != 0])]

path = "../results/Results_Reports/"
plt.plot(thresholds, precisions, label="Precision")
plt.plot(thresholds, recalls, label="Recall")
plt.axvline(threshold, color="black", alpha=0.5)
plt.title('Precision and Recall for different thresholds')
plt.xlabel('Threshold')
plt.ylabel('Precision/Recall')
plt.legend()
plt.savefig(os.path.join(path, "Precision_vs_Recall.png"),dpi=600)
plt.close()

if True:#"train_{}_normal".format(group) in globals().keys():
    y_pred = (y_pred > threshold).astype(np.int64)
    test_class = "normal"
else:
    #y_true = 1 - y_true
    y_pred = (y_pred < threshold).astype(np.int64)
    test_class = "abnormal"
#y_pred = (y_pred > threshold).astype(np.int64)

TP = tf.keras.metrics.TruePositives()
TN = tf.keras.metrics.TrueNegatives()
FP = tf.keras.metrics.FalsePositives()
FN = tf.keras.metrics.FalseNegatives()
AUC = tf.keras.metrics.AUC()

TP.update_state(y_true, y_pred)
TN.update_state(y_true, y_pred)
FP.update_state(y_true, y_pred)
FN.update_state(y_true, y_pred)
AUC.update_state(y_true, y_pred)

print("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(
    threshold,
    accuracy(TP.result().numpy(), TN.result().numpy(), FP.result().numpy(), FN.result().numpy()),
    precision(TP.result().numpy(), FP.result().numpy()),
    TN.result().numpy() / (TN.result().numpy() + FN.result().numpy()),
    recall(TP.result().numpy(), FN.result().numpy()),
    specificity(TN.result().numpy(), FP.result().numpy()),
    f1_score(TP.result().numpy(), FP.result().numpy(), FN.result().numpy()),
    AUC.result().numpy()
))

0.3996993503977905	0.7063252925872803	0.6915584206581116	0.7191011309623718	0.6805111765861511	0.7293447256088257	0.6859903381642513	0.7049279808998108


## Qualitative metrics

### Errors exploration

In [22]:
data_table = []
data_columns = ["Exp ID", "Min", "Max", "Mean", "Std", "Ske", "Kur", "CDF (x > 0)"]

for g in ["train", "test", "all"]:
        if g == "train":
            classes = ["check"]
        else:
            classes = ["normal", "abnormal"]
        for cl in classes:
            for t in ["encoder", "contextual", "adversarial"]:
                
                if cl == "check":
                    data_table.append(["{} {}".format(g, t)] + [None]*5)
                else:
                    data_table.append(["{} {} {}".format(g, t, cl)] + [None]*5)
                    
                for exp_id in experiments_ids:
                    if cl == "check":
                        if "train_{}_normal".format(t) in exps[exp_id].keys():
                            c = "normal"
                        else:
                            c = "abnormal"
                    else:
                        c = cl
                    
                    data = exps[exp_id]["{}_{}_{}".format(g, t, c)]
                    m = np.mean(data)
                    s = np.std(data)
                
                    data_table.append([
                        exp_id,
                        np.min(data),
                        np.max(data),
                        m,
                        s,
                        stats.skew(data),
                        stats.kurtosis(data),
                        1 - stats.norm(m, s).cdf(0)
                    ])
pd.DataFrame(data_table, columns=data_columns).to_excel(
    os.path.join(save_path_2_excels, "qualitative_metrics.xlsx"), 
    index=False
)

### Normality tests

In [23]:
data_table = []
data_columns = ["Exp ID", "Brown", "Levene", "Barlett", "DAP", "AD", "SW", "Chi^2", "F test"]

for g in ["train", "test", "all"]:
    if g == "train":
        classes = ["check"]
    else:
        classes = ["normal", "abnormal"]
    for cl in classes:
        for t in ["encoder", "contextual", "adversarial"]:

            if cl == "check":
                data_table.append(["{} {}".format(g, t)] + [None]*8)
            else:
                data_table.append(["{} {} {}".format(g, t, cl)] + [None]*8)

            for exp_id in experiments_ids:
                if cl == "check":
                    if "train_{}_normal".format(t) in exps[exp_id].keys():
                        c = "normal"
                    else:
                        c = "abnormal"
                else:
                    c = cl

                data = sorted(exps[exp_id]["{}_{}_{}".format(g, t, c)])
                norm_dist = sorted(stats.norm.rvs(loc=np.mean(data), scale=np.std(data), size=len(data), random_state=8128))
                chi_test = chiSquare_test(data, norm_dist)
                data_table.append([
                    exp_id,
                    int(brownForsythe_test(data, norm_dist)),
                    int(levene_test(data, norm_dist)),
                    int(bartlett_test(data, norm_dist)),
                    int(dagostinoPearson_test(data)),
                    int(andersonDarling_test(data)),
                    int(shapiroWilks_test(data)),
                    "{} ({})".format(int(chi_test[0]), round(chi_test[1], 5)),
                    int(fOneWay_test(data, norm_dist))
                ])
pd.DataFrame(data_table, columns=data_columns).to_excel(
    os.path.join(save_path_2_excels, "normality_tests.xlsx"), 
    index=False
)

### Grouping tests

In [24]:
data_table = []
data_columns = ["Exp ID", "Brown", "Levene", "Barlett", "MW", "KW", "KS", "F test"]

for t in ["encoder", "contextual", "adversarial"]:
    for g1, g2 in [("train", "test"), ("test", "all"), ("train", "all")]:
        data_table.append(["{} {} vs {}".format(t, g1, g2)] + [None]*7)
        for exp_id in experiments_ids:
            if "train_{}_normal".format(t) in exps[exp_id].keys():
                c = "normal"
            else:
                c = "abnormal"
                
            data1 = sorted(exps[exp_id]["{}_{}_{}".format(g1, t, c)])
            data2 = sorted(exps[exp_id]["{}_{}_{}".format(g2, t, c)])
            
            data_table.append([
                exp_id,
                int(brownForsythe_test(data1, data2)),
                int(levene_test(data1, data2)),
                int(bartlett_test(data1, data2)),
                int(mannWhitney_test(data1, data2)),
                int(kruskalWallis_test(data1, data2)),
                int(kolmogorovSmirnov_test(data1, data2)),
                int(fOneWay_test(data1, data2))
            ])
pd.DataFrame(data_table, columns=data_columns).to_excel(
    os.path.join(save_path_2_excels, "grouping_tests.xlsx"), 
    index=False
)

### Classing tests

In [26]:
data_table = []
data_columns = ["Exp ID", "Brown", "Levene", "Barlett", "Chi^2 N->A", "Chi^2 A->N", "MW", "KW", "KS", "F test"]

for g in ["test", "all"]:
    for t in ["encoder", "contextual", "adversarial"]:
        data_table.append(["{} {}".format(g, t)] + [None]*9)
        for exp_id in experiments_ids:
            data1 = sorted(exps[exp_id]["{}_{}_normal".format(g, t)])
            data2 = sorted(exps[exp_id]["{}_{}_abnormal".format(g, t)])
            
            row = [
                exp_id,
                int(brownForsythe_test(data1, data2)),
                int(levene_test(data1, data2)),
                int(bartlett_test(data1, data2)),
            ]
            if g == "test":
                row += [
                    int(mannWhitney_test(data1, data2)),
                    int(kruskalWallis_test(data1, data2)),
                    int(kolmogorovSmirnov_test(data1, data2)),
                    int(fOneWay_test(data1, data2)),
                    None,
                    None
                ]
            else:
                chi_1_test = chiSquare_test(data1, data2)
                chi_2_test = chiSquare_test(data2, data1)
                row += [
                    "{} ({})".format(int(chi_1_test[0]), round(chi_1_test[1], 5)),
                    "{} ({})".format(int(chi_2_test[0]), round(chi_2_test[1], 5)),
                    int(mannWhitney_test(data1, data2)),
                    int(kruskalWallis_test(data1, data2)),
                    int(kolmogorovSmirnov_test(data1, data2)),
                    int(fOneWay_test(data1, data2))
                ]
            
            data_table.append(row)
        
pd.DataFrame(data_table, columns=data_columns).to_excel(
    os.path.join(save_path_2_excels, "classing_tests.xlsx"), 
    index=False
)

In [57]:
# Normality test for total data
for c in ["normal", "abnormal"]:
    for t in ["encoder", "contextual", "adversarial"]:
        data = sorted(globals()["total_{}_{}".format(t, c)])
        norm_dist = sorted(stats.norm.rvs(loc=np.mean(data), scale=np.std(data), size=len(data), random_state=8128))
        chi_test = chiSquare_test(data, norm_dist)
        print("{} tests ({}):\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(
            t,
            c,
            int(brownForsythe_test(data, norm_dist)),
            int(levene_test(data, norm_dist)),
            int(bartlett_test(data, norm_dist)),
            int(dagostinoPearson_test(data)),
            int(andersonDarling_test(data)),
            int(shapiroWilks_test(data)),
            "{} ({})".format(int(chi_test[0]), round(chi_test[1], 5)),
            int(fOneWay_test(data, norm_dist))
        ))
        print("")

encoder tests (normal):	1	0	1	0	0	0	1 (26.59235)	1

contextual tests (normal):	1	1	1	0	0	0	1 (2.91873)	1

adversarial tests (normal):	0	0	1	0	0	0	1 (56.9364)	1

encoder tests (abnormal):	0	1	1	0	0	0	1 (75.33484)	1

contextual tests (abnormal):	1	1	1	1	0	0	1 (0.84015)	1

adversarial tests (abnormal):	1	1	1	0	0	0	1 (12.09595)	1



In [ ]:
# Grouping test for total data 
for t in ["encoder", "contextual", "adversarial"]:
    print("-------------- {} ---------------------".format(t))
    for g1, g2 in [("train", "test"), ("test", "all"), ("train", "all")]:
        if "train_{}_normal".format(t) in globals().keys():
            c = "normal"
        else:
            c = "abnormal"
        data1 = sorted(globals()["{}_{}_{}".format(g1, t, c)])
        data2 = sorted(globals()["{}_{}_{}".format(g2, t, c)])
        print("{} vs {}: {}\t{}\t{}\t{}\t{}\t{}\t{}".format(
            g1,
            g2,
            int(brownForsythe_test(data1, data2)),
            int(levene_test(data1, data2)),
            int(bartlett_test(data1, data2)),
            int(mannWhitney_test(data1, data2)),
            int(kruskalWallis_test(data1, data2)),
            int(kolmogorovSmirnov_test(data1, data2)),
            int(fOneWay_test(data1, data2))
        ))
        print("")

## Visual results

### Loading latent vectors

In [25]:
for exp_id in experiments_ids:
    base_path = os.path.join(exps[exp_id]["path"], "outputs/latent_vectors/")
    for n in ["generator", "discriminator"]:
        for t in ["input", "output"]:
            for c in ["normal", "abnormal"]:
                exps[exp_id]["all_{}_{}_{}".format(n, t, c)] = []
    
    for n in ["generator", "discriminator"]:
        for t in ["input", "output"]:
            for m in ["train", "test"]:
                if m == "train":
                    if len(os.listdir(os.path.join(base_path, t + "_" + n, m, "normal"))) != 0:
                        classes = ["normal"] 
                    elif len(os.listdir(os.path.join(base_path, t + "_" + n, m, "abnormal"))) != 0:
                        classes = ["abnormal"]
                    else:
                        raise AssertionError("There are no vectors for the train folders in the experiment {}".format(exp_id))
                else:
                    classes = ["normal", "abnormal"]
                for c in classes:
                    all_data = "all_{}_{}_{}".format(n, t, c)
                    data = "{}_{}_{}_{}".format(m, n, t, c)
                    exps[exp_id][data] = []
                    path = os.path.join(base_path, t + "_" + n, m, c)
                    for file in sorted(os.listdir(path)):
                        vector = np.load(os.path.join(path, file))
                        exps[exp_id][data].append(vector)
                        exps[exp_id][all_data].append(vector)
                    exps[exp_id][data] = np.r_[exps[exp_id][data]]